## **1. KcELECTRA 모델 불러오기**

In [ ]:
import torch

In [ ]:
print(torch.__version__)

1.13.1+cu116


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    import transformers, emoji, soynlp, pytorch_lightning
except:
    !pip install -U -q transformers emoji soynlp pytorch-lightning

In [ ]:
import os
import pandas as pd

from pprint import pprint


import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR, CosineAnnealingWarmRestarts

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [ ]:
args = {
    'random_seed': 42, # Random Seed
    'pretrained_model': 'beomi/KcELECTRA-base',  # Transformers PLM name
    'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    'batch_size': 32,
    'lr': 5e-6,  # Starting Learning Rate
    'epochs': 300,  # Max Epochs
    'max_length': 64,  # Max Length input size
    'train_data_path': '/content/drive/MyDrive/train_6000_2.csv',  # Train Dataset file 
    'val_data_path': '/content/drive/MyDrive/valid_6000_2.csv',  # Validation Dataset file 
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',  # AdamW vs AdamP
    'lr_scheduler': 'exp',  # ExponentialLR vs CosineAnnealingWarmRestarts
    'fp16': True,  # Enable train on FP16(if GPU)
    'tpu_cores': 0,  # Enable TPU with 1 core or 8 cores
    # 'cpu_workers': os.cpu_count(),
    'cpu_workers': 0
}

In [ ]:
accuracy_list = []
loss_list = []
class Model(LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters() # 이 부분에서 self.hparams에 위 kwargs가 저장된다.
        
        self.clsfier = AutoModelForSequenceClassification.from_pretrained(self.hparams.pretrained_model, num_labels = 5)
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.hparams.pretrained_tokenizer
            if self.hparams.pretrained_tokenizer
            else self.hparams.pretrained_model
        )

    def forward(self, **kwargs):
        return self.clsfier(**kwargs)

    def step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def epoch_end(self, outputs, state='train'):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
        
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average = 'micro')
        rec = recall_score(y_true, y_pred, average = 'micro')
        f1 = f1_score(y_true, y_pred, average = 'micro')

        self.log(state+'_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)
    
        accuracy_list.append(acc)
        loss_list.append(loss)
        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        return {'loss': loss}
    
    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state='train')

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state='val')

    def configure_optimizers(self):
        if self.hparams.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.hparams.lr, weight_decay = 0.3)
            # optimizer = AdamW(self.parameters(), lr=0)#0에 가까운 아주 작은 값을 입력해야함
        elif self.hparams.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.hparams.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.hparams.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.hparams.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        elif self.hparams.lr_scheduler == 'cosup':
            scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=10, T_mult=2, eta_max=0.1,  T_up=10, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def clean(self, x):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x

    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            self.clean(str(x)),
            padding='max_length',
            max_length=self.hparams.max_length,
            truncation=True,
            **kwargs,
        )

    def preprocess_dataframe(self, df):
        df['document'] = df['document'].map(self.encode)
        return df

    def dataloader(self, path, shuffle=False):
        df = self.read_data(path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.hparams.batch_size * 1 if not self.hparams.tpu_cores else self.hparams.tpu_cores,
            shuffle=shuffle,
            num_workers=self.hparams.cpu_workers,
        )

    def train_dataloader(self):
        return self.dataloader(self.hparams.train_data_path, shuffle=True)

    def val_dataloader(self):
        return self.dataloader(self.hparams.val_data_path, shuffle=False)

In [ ]:
#모델 불러오기
model1 = torch.load('/content/drive/MyDrive/NLP Project/kcelectra_total_model.pt')
model1.eval()

Model(
  (clsfier): ElectraForSequenceClassification(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(50135, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0): ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): ElectraSelfOutput(
                (dense): Linear(in_features=768, out_f

## **2. 일기 문장 분리**

In [ ]:
pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.1-py3-none-any.whl size=53218 sha256=87f79fbaace4c38158bb95002fb9a92a247c848f86d50fbd0655947593ff147c
  Stored in directory: /root/.cache/pip/wheels/8c/f9/05/a93eb0defc9eed494ee4d42e3d9076fade98f08982f41cf09f
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646667 sha256=7b79b2fa6eeac230df0eb1d98b3d394b541e29c0fec4378f1acef6420441ba3a
  Stored in directory: /root/.cache/pip/wheels/a9/98/a3/085a909aa25805f321467a3af3b80d43b6094ed335b4818775
Successfully built kss pecab
  Attempting uninstall: emoji
    Found

In [ ]:
!pip uninstall emoji
!pip install emoji==1.2.0

Found existing installation: emoji 1.2.0
Uninstalling emoji-1.2.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/emoji-1.2.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/emoji/*
Proceed (Y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(emoji.__version__)

1.2.0


In [ ]:
import kss

In [ ]:
#일기 입력
text = "정말 옛날옛날에 애용하던 러쉬에 오랜만에 가봤다. 다시 보니 또 추억 뿜뿜 몇 가지 들고 집으로 왔다:) 거의 매달 색을 바꾸니 헤어가 남아나질 않...ㅠ 하이라이트를 다음엔 꼭 하리라!ㅋ 혼자만의 시간을 보낸 론 헤르만 카페 정말 너무 예쁘고, 맛있는 곳완전 추천추천!! 유치원 갔다오는 회장님과 함께 또 돈카츠 먹기ㅎㅎ 맛있는 점심을 꼬기꼬기 먹고, 꼬기 사진은 유툽용으로 동영상만 찍었더니 하나도 없다ㅠ 주변 공원으로 가서 회장님 자전거 연습:) 오늘이 추석이라는 사실도 잊고 있었다.. 운동 간단히 하고 나오는데, 보름달이 너무 예뻤다:)"
text1 = "나는 자책을 많이 하는 편이다. 무엇을 해도 끝날 때 미련과 아쉬움을 느끼는 게 사람 아니겠는가. 난 항상 내가 하는 일에 아쉬움과 미련이 남는다. 글을 써서 제출할 때도 괜히 한번 더 확인하고 그림을 낼 때도 덧칠도 하고 왜 내가 하는 것은 다 완벽하지 않는가 싶다. 처음에 내가 완벽하다고 생각한 것도 내 생각이고 객관적인 평가에서 떨어질 때 완벽하다고 생각했지만 남의 평가에 괜히 아쉬움과 죄책감이 남는다. 내가 좀 더 잘했더라면 조금 더 좋은 결과를 가져 오지 않았을까 생각하며 혼자 우울해 한다. 다른 사람들에게 기쁜 모습만 보여주고 싶다. 그런데 계속 혼자 속에 품기 답답하여 많은 사람들이 있는 곳에 글을 써본다. 잘못을 하여서 혼이 날 때도 내가 잘못한 것을 알지만 인정하기 싫은 것이다. 하지만 인정을 하지 않을 때도 내가 이렇게 저렇게 하였으면 어땠을지 과거의 나를 자책하고도 한다. 이렇게 혼자 울 때, 나 혼자 모든 것을 해결하려고 할 때 나는 내 스스로가 싫다."
text2 = "셋째 주는 특별한 일이 없이 옷정리에 냉장고 청소에 책장 다꺼내서 닦고 대청소의 주간이었다. 저녁에는 티비 보면서 조촐하게 맥주나 막걸리도 한잔하고. 요즘 매일 아침에 일어나면 책을 읽는다. 계속 어려운 책만 읽다가 소설을 읽으니 너무 재밌었다. 2010년 우연히 카지노를 읽게 되면서 김진명 작가님의 소설에 빠져들었다. 어쩜 이렇게 하나같이 다 흥미진진하고 재밌는지! 우울하거나 마음이 허한 날은 시집을 읽는데 이건 나에게 꼭 해주고싶은 말이다. 그리고 드디어 기다리던 바리스타 2급 실기 합격 문자가 왔다. 실수 하나 때문에 일주일 내내 신경 쓰이고 무기력했는데 그래도 너무 기쁘다. 오랜만에 설레고 재밌는 경험이었다. 시장에 가서 잔치국수도 먹고 과일이랑 야채도 잔뜩 사왔다. 갈때마다 사먹는 3천원짜리 국수인데 멸치육수를 대량으로 끓이니 국물이 엄청 진하고 시원하다. 2천원 주고 작은 고구마 한봉지 사와서 난로에 구워먹었다. 군고구마 냄새가 정말 좋다."
text3 = "문득 첫 회사에서 생일을 맞이했을 때가 생각났다. 엄마가 꽃바구니를 회사로 보내주셨는데 아마도 그때 카드에 적힌 말에 '내일 당장 그만두더라도 평생 다닐 것처럼 열심히 해라'같은 종류의 말씀이 적혀있었다. 그렇게 내가 엄마의 바람대로 맡은 일에 최선을 다하고 내가 했던 선택에 대해서 끈기를 가지고 책임을 지는 사람으로 살아왔다면 좋았을걸. 이제 와서 후회해봤자 달라질 것은 없지만 후회가 된다. 다시는 만나지 못할 것 같았던 사람의 연락을 받았고 그로인해 어떤 미래가 펼쳐질지 모르겠다. 왜 그랬을까...? 우선 도망가고 싶은 마음이 컸다. 내가 선택할 수 없는 것들은 집착하지 말고 흘러가게 내버려둬야지. 늘 머리로는 알면서도 갑자기 이렇게 훅 치고 들어오는 사건에는 동요하게 된다. 마음 정리를 잘 하자. 잘 비우자. 내가 할 수 있는 것들만 잘하자. 이렇게 아무리 마음을 다잡아도 내 마음대로 안되는 것도 같지만 그래도 아무 준비도 없이 닥치는 상황보다는 낫지 않겠나? 그렇기에 인간이고, 그럼에도 노력하는 것이다. 여러모로 고민도, 걱정도 많은 하루다."
text4 = "뭐 많이 먹지도 않았는데 증량이라 속상하다. 왜지..? 어제 저녁에 먹은 만두 3개 때문인가.......?? 아침부터 큰 증량이라 기분이 별로였다. 주말에 샐러드는 안 먹었지만 나름 절제해서 식단 잘 지켰다고 생각했는데 아니었나보다. 아니면 국물류를 너무 먹은게 문제였을까? 이런저런 반찬들이 문제였을까? 정말 고기!!만 먹어야 하나? 아침부터 1키로 때문에 속상한 상태로 출근준비를 했다. 1키로 증량이라니 너무하는거 아닙니꽈?????!! (점심에 계속 부대찌개 먹고 싶어서 고민많이 했는데 증량한거 생각하며 꾹꾹 참고 엄마가 싸준 나물들 알뜰하게 넣고 나물 비빔밥 먹었다. 이제 마야 고추장과 들기름, 나물을 냉장고에 넣어두고 밥이랑 고기를 싸와서 비벼 먹어 보려고 도전했는데 생각보다 성공적이었다. 비빔밥은 좀 차갑게 먹어도 괜찮기에 ㅎㅎㅎㅎ 신나게 먹어줬다. 내일은 좀 더!! 클린할 수 있기를...."
text5 = "참 재미없었던 한 주였다. 날씨를 크게 의식하는 편도 아닌데 우중충한 날씨처럼 썩 내키지 않는 일이 많았다. 코로나기간 동안 핑계대며 친구들을 피해다녔는데 결국은 피할 수 없는 순간이 오는 것 같다. 아마도 4년? 만에 친구를 만난것 같은데.. 친구들에 비해 나 자신이 너무 초라해보여서 말이 턱턱 막혀서, 말 꺼내기 힘들었다. 거의 20년 지기 친구인데 속사정 말하기가 좀 창피했다. 결국 나는 스스로를 실패자로 정의해서일까. 그리고 나서 또 거짓말처럼 다른 친구들한테 계속해서 연락이 왔다. 사실 코로나를 핑계로 상황을 부정하고픈걸까. 연락해주는게 정말 고마운데 내 마음의 여유가 하나도 없구나. 와인 마실 기분이 아니라 시키진 않았는데 궁금해서 지금도 마셔보고싶긴하다. 회사사람하고 다툰날도 있었다. 3년동안 회사에 만족할 수 있었던건 사람들이 다들 좋아서 였는데, 뭔가 뒷통수를 세게 맞은듯한 느낌이 들어서 따졌다. 이번주부터 자격증 공부를 시작했는데 생각이상으로 너무너무 재미없고 무슨 말을 하는지 이해가 안된다. 이번주는 모임이 많아서, 목표 공부량을 못채웠는데 과연 할 수 있을지 의문이다."
text6 = "이번주가.. 어떻게 지나갔는지 잘 모르겠다 일하고 먹고 자고.. 그냥 일상의 반복이었는데 짜증이 많이 났던 일주일이었던 것 같다. 긍정적으로 생각하자는 마음도 물론 생각하고 있지만 말도 부정적으로 나오고 다른 사람의 말도 삐딱하게 받아들이고 스스로 스트레스를 많이 받는다고 생각했다. 항상 하던 일들이 갑자기 틀어질 때 엄청난 스트레스를 받는데 그게 내가 아닌 다른 사람들에게 영향을 주는 일이 되기 때문에 빠르게 정상화를 시키기 위해 에너지 소모를 많이 한다. 지나고 보면 결과적으로 잘 풀리긴 하는데.. 그 과정에서 너무 신경을 많이 쓴다. 매일 쓴 일기를 돌아보면 감사한 일도 매일 있었는데 역시 마무리가 좋아야 좋은 기억으로 남는다.. 금요일이 좀 일진이 안 좋은 하루였던 것 같다. 아침부터 정신이 없고 오후에도, 퇴근시간에도 자꾸 일이 발생했다. 언제나 예기치 못한 일은 발생할 수 있고 처리하는 건 그렇게 어려운 일이 아니라고 생각하고 싶은데 잘 안됐다. 스트레스를 먹는 걸로 푸는데.. 아이스크림을 일주일의 절반은 매일 먹었고 하루에 제일 많이 먹은 날은 4개를 먹은 것 같다. 음식 말고 스트레스를 푸는 법은 드라마를 보는 것이다. 모범형사2를 정주행했는데 역시 드라마는 안 끊기고 봐야 한다. 모범형사1을 재미있게 봐서 몰아서 보려고 끝날 때까지 기다렸는데 이번 주에 몰아서 봤다. 배우들이 다들 연기를 진짜 잘해서 드라마 보는데 같이 화가 나고 몰입도 잘 됨. 김효진 배우님이 그렇게 미친 사람 연기를 잘 하실 줄 몰랐는데.. 최고예요 "
text7 = "학교에서 수업을 마치고 나면 쉬는시간이 찾아온다.학생들은 쉬는시간에 여러가지 활동들을 한다. 쉬는시간을 즐겁게 보내는 방법은 다양하지만, 개인적인 의견으로 난 수다를 떠는 것이 가장 즐겁다.자기가 말하고 싶었던 것을 친한친구들에게 말해줄 수도 있고, 힘들었던 일이나 화가 났던 일을 친구들에게 털어놓을 수도 있다. 그러므로 난 쉬는시간에 수다를 떠는 것이 가장 즐겁다고 생각한다."
text8 = "나는 요즘 해야 할 것을 하기가 귀찮아지기 시작했다. 해야 할 것을 하고 싶은 마음이 생길 때는 썩 좋지 못한 일이 생겨 짜증이 나고 감정이 북받친다. 조금 좋지 않은 일을 겪어도 의욕이 바로 사라지고 감정적인 사람이 되는 것은, 내가 상처를 쉽게 받기 때문인 것 같다. 사람들은 흔히 이렇게 상처를 쉽게 받는 사람을 유리 멘탈이라 일컫는다. 결국 다시 처음의 단계로 돌아가게 된다. 다음에 더 잘하겠다고 다짐을 해도, 전례와 같은 일이 반복된다. 그래서, 내가 느끼기에는 할 것은 제대로 하지 않고 놀기만 하는 것 같다. 이럴 때는 너무 후회가 된다. 나는 귀찮음과 유리 멘탈에서 벗어나고 싶지만, 뜻대로 되지 않아 나 자신이 답답하기도 하다. 나는 그럴 때마다 공부해야 하는 시간을 허비해서 나의 미래에 지장이 있을까 걱정이 된다."
text9 = "나는 행복해. 정말로 행복해. 너무 행복해"
sp_text = kss.split_sentences(text6)

In [ ]:
sp_text

['이번주가..',
 '어떻게 지나갔는지 잘 모르겠다 일하고 먹고 자고..',
 '그냥 일상의 반복이었는데 짜증이 많이 났던 일주일이었던 것 같다.',
 '긍정적으로 생각하자는 마음도 물론 생각하고 있지만 말도 부정적으로 나오고 다른 사람의 말도 삐딱하게 받아들이고 스스로 스트레스를 많이 받는다고 생각했다.',
 '항상 하던 일들이 갑자기 틀어질 때 엄청난 스트레스를 받는데 그게 내가 아닌 다른 사람들에게 영향을 주는 일이 되기 때문에 빠르게 정상화를 시키기 위해 에너지 소모를 많이 한다.',
 '지나고 보면 결과적으로 잘 풀리긴 하는데..',
 '그 과정에서 너무 신경을 많이 쓴다.',
 '매일 쓴 일기를 돌아보면 감사한 일도 매일 있었는데 역시 마무리가 좋아야 좋은 기억으로 남는다..',
 '금요일이 좀 일진이 안 좋은 하루였던 것 같다.',
 '아침부터 정신이 없고 오후에도, 퇴근시간에도 자꾸 일이 발생했다.',
 '언제나 예기치 못한 일은 발생할 수 있고 처리하는 건 그렇게 어려운 일이 아니라고 생각하고 싶은데 잘 안됐다.',
 '스트레스를 먹는 걸로 푸는데.. 아이스크림을 일주일의 절반은 매일 먹었고 하루에 제일 많이 먹은 날은 4개를 먹은 것 같다.',
 '음식 말고 스트레스를 푸는 법은 드라마를 보는 것이다.',
 '모범형사2를 정주행했는데 역시 드라마는 안 끊기고 봐야 한다.',
 '모범형사1을 재미있게 봐서 몰아서 보려고 끝날 때까지 기다렸는데 이번 주에 몰아서 봤다.',
 '배우들이 다들 연기를 진짜 잘해서 드라마 보는데 같이 화가 나고 몰입도 잘 됨.',
 '김효진 배우님이 그렇게 미친 사람 연기를 잘 하실 줄 몰랐는데..',
 '최고예요']

## **3. 모델 추론과정**

In [ ]:
def infer(x):
    return torch.softmax(
        model1(**model1.tokenizer(x, return_tensors='pt')
    ).logits, dim=-1)

In [ ]:
#일기 데이터 한 문장씩 모델로 돌리기
#방법 1. 감정별 리스트 만들고 / 문장 돌아갈때 마다 각 감정에 해당하는 값들 추가 -> 문장이 상대적이어서 패스
#방법 2. 각 문장의 최대 감정을 구함(max) -> 구한 최대 감정들을 count해서 가장 많이나온 감정!
import numpy as np
def emotion_count(li):
  counts = {}
  for x in li:
    if x in counts:
      counts[x] += 1
    else:
      counts[x] = 1
  return counts

def top_3(count_dict, n = 3):
  return sorted(count_dict.items(), reverse = True, key = lambda x:x[1])[:n]

def neu_emo(text):
  sec_emotions = []
  for sent in sp_text:
    single_text = infer(sent)

    #single_text에 대한 softmax값이 담길 lo리스트
    lo = []
    #tensor형태 -> 1차원 list로 변경
    for i in single_text:
      a = i.detach().numpy()
      a = a.tolist()# float type
      lo.append(a)
      lo = np.concatenate(lo).tolist()#2차원 배열 -> 1차원 배열로
      # print(lo)
      lo_sor = sorted(lo)
      sec_max_emo = lo.index(lo_sor[-2])#두번째로 큰 값의 인덱스 저장
      # print(sec_max_emo)

      if sec_max_emo == 0:
        sec_emotions.append("행복")
      elif sec_max_emo == 1:
        sec_emotions.append("중립")
      elif sec_max_emo == 2:
        sec_emotions.append("불안")
      elif sec_max_emo == 3:
        sec_emotions.append("슬픔")
      elif sec_max_emo == 4:
        sec_emotions.append("분노")    

    # print("max emotions list : ", max_emotions)

  print("max emotions list : ", sec_emotions)
  counts = emotion_count(sec_emotions)
  print("emotion counts: ", counts)

  top = top_3(counts, n = 3)
  # first_emo_cnt = top[0] 
  return top



#최대 감정들이 담길 리스트
max_emotions = []
final_emo = []

#문장별로 infer()돌리기
for sent in sp_text:
  single_text = infer(sent)

  #single_text에 대한 softmax값이 담길 lo리스트
  lo = []
  #tensor형태 -> 1차원 list로 변경
  for i in single_text:
    a = i.detach().numpy()
    a = a.tolist()# float type
    lo.append(a)
    lo = np.concatenate(lo).tolist()#2차원 배열 -> 1차원 배열로
    # print(lo)
    max_emo = lo.index(max(lo))#최대값 = 최대감정 인덱스
    # print(max_emo)

    if max_emo == 0:
      max_emotions.append("행복")
    elif max_emo == 1:
      max_emotions.append("중립")
    elif max_emo == 2:
      max_emotions.append("불안")
    elif max_emo == 3:
      max_emotions.append("슬픔")
    elif max_emo == 4:
      max_emotions.append("분노")    

  # print("max emotions list : ", max_emotions)

print("max emotions list : ", max_emotions)
counts = emotion_count(max_emotions)
print("emotion counts: ", counts)

top = top_3(counts, n = 3)
print(top)

first_emo = top[0][0]
if len(top) == 1:
  if first_emo == '중립': #top_3에 중립만 나올 때 -> 문장 별 2순위 감정까지 넣기
    top = neu_emo(sp_text)#top 재설정해주기
    print(top)
    first_emo = top[0][0]#first_emo 재정립
    if len(top) == 1:#neu_emo돌려도 값 하나일 때
      # print(first_emo)
      final_emo.append(first_emo)
  else:
    print(first_emo)#top_3에 한가지 감정만 나올 때
    final_emo.append(first_emo)


if len(top) == 2:
  second_emo = top[1][0]
  final_emo.append(first_emo)
  final_emo.append(second_emo)
  print(first_emo, second_emo)
  
elif len(top) == 3:
  second_emo = top[1][0]
  third_emo = top[2][0]
  final_emo.append(first_emo)
  final_emo.append(second_emo)
  final_emo.append(third_emo)
  print(first_emo, second_emo, third_emo)

max emotions list :  ['중립', '중립', '중립', '중립', '중립']
emotion counts:  {'중립': 5}
[('중립', 5)]
max emotions list :  ['불안', '불안', '행복', '행복', '행복']
emotion counts:  {'불안': 2, '행복': 3}
[('행복', 3), ('불안', 2)]
행복 불안


In [ ]:
import numpy as np
def emotion_count(li):
  counts = {}
  for x in li:
    if x in counts:
      counts[x] += 1
    else:
      counts[x] = 1
  return counts

def top_3(count_dict, n = 3):
  return sorted(count_dict.items(), reverse = True, key = lambda x:x[1])[:n]

def neu_emo(text):
  sec_emotions = []
  for sent in sp_text:
    single_text = infer(sent)

    lo = []

    for i in single_text:
      a = i.detach().numpy()
      a = a.tolist()
      lo.append(a)
      lo = np.concatenate(lo).tolist()
      lo_sor = sorted(lo)
      sec_max_emo = lo.index(lo_sor[-2])

      if sec_max_emo == 0:
        sec_emotions.append("행복")
      elif sec_max_emo == 1:
        sec_emotions.append("중립")
      elif sec_max_emo == 2:
        sec_emotions.append("불안")
      elif sec_max_emo == 3:
        sec_emotions.append("슬픔")
      elif sec_max_emo == 4:
        sec_emotions.append("분노")    

  counts = emotion_count(sec_emotions)
  top = top_3(counts, n = 3)
  return top
#시작
max_emotions = []
for sent in sp_text:
  single_text = infer(sent)
  lo = []

  for i in single_text:
    a = i.detach().numpy()
    a = a.tolist()
    lo.append(a)
    lo = np.concatenate(lo).tolist()
    max_emo = lo.index(max(lo))

    if max_emo == 0:
      max_emotions.append("행복")
    elif max_emo == 1:
      max_emotions.append("중립")
    elif max_emo == 2:
      max_emotions.append("불안")
    elif max_emo == 3:
      max_emotions.append("슬픔")
    elif max_emo == 4:
      max_emotions.append("분노")    

counts = emotion_count(max_emotions)
top = top_3(counts, n = 3)

first_emo = top[0][0]
if len(top) == 1:
  if first_emo == '중립':
    top = neu_emo(sp_text)
    first_emo = top[0][0]
  else:
    print(first_emo)

if len(top) == 2:
  second_emo = top[1][0]
  print(first_emo, second_emo)
  
elif len(top) == 3:
  second_emo = top[1][0]
  third_emo = top[2][0]
  print(first_emo, second_emo, third_emo)